# PGP Encryption Implementation in Python

- This implementation is performed on Windows 10
- The `pgpy` module is built on top of *OpenPGP*
- Must first `pip install PGPy`

## Generating keys

In [1]:
import pgpy
from pgpy.constants import PubKeyAlgorithm, KeyFlags, HashAlgorithm, SymmetricKeyAlgorithm, CompressionAlgorithm

In [2]:
help(pgpy.PGPKey.new)

Help on method new in module pgpy.pgp:

new(key_algorithm, key_size, created=None) method of abc.ABCMeta instance
    Generate a new PGP key
    
    :param key_algorithm: Key algorithm to use.
    :type key_algorithm: A :py:obj:`~constants.PubKeyAlgorithm`
    :param key_size: Key size in bits, unless `key_algorithm` is :py:obj:`~constants.PubKeyAlgorithm.ECDSA` or
           :py:obj:`~constants.PubKeyAlgorithm.ECDH`, in which case it should be the Curve OID to use.
    :type key_size: ``int`` or :py:obj:`~constants.EllipticCurveOID`
    
    :param created: When was the key created? (None or unset means now)
    :type created: :py:obj:`~datetime.datetime` or None
    :return: A newly generated :py:obj:`PGPKey`



In [3]:
# generate an RSA primary key
# 4096 is the key size in bits 
# unless key_algorithm is PubKeyAlorithm.ECDSA or .ECDH, then it's the Curve OID
key = pgpy.PGPKey.new(key_algorithm=PubKeyAlgorithm.RSAEncryptOrSign, key_size=4096)

In [4]:
#help(key)

- The new key is not usable without any user ID's
- Must create and add user ID's to the key

In [5]:
help(pgpy.PGPUID.new)

Help on method new in module pgpy.pgp:

new(pn, comment='', email='') method of builtins.type instance
    Create a new User ID or photo.
    
    :param pn: User ID name, or photo. If this is a ``bytearray``, it will be loaded as a photo.
               Otherwise, it will be used as the name field for a User ID.
    :type pn: ``bytearray``, ``str``, ``unicode``
    :param comment: The comment field for a User ID. Ignored if this is a photo.
    :type comment: ``str``, ``unicode``
    :param email: The email address field for a User ID. Ignored if this is a photo.
    :type email: ``str``, ``unicode``
    :returns: :py:obj:`PGPUID`



In [6]:
# create a new user id for the key we created
# pn is the username or a bytearray for an image
# email is the email of the user, ignore if pn is an image
uid = pgpy.PGPUID.new(pn='Chuck Tucker', comment='SC side username', email='chuck.tucker@santeecooper.com')

- Must add the user ID(s) to the key
- This is where preferences are set
- Until we do this, there are no built-in key preferences

In [7]:
help(key.add_uid)

Help on method add_uid in module pgpy.pgp:

add_uid(uid, selfsign=True, **prefs) method of pgpy.pgp.PGPKey instance
    Add a User ID to this key.
    
    :param uid: The user id to add
    :type uid: :py:obj:`~pgpy.PGPUID`
    :param selfsign: Whether or not to self-sign the user id before adding it
    :type selfsign: ``bool``
    
    Valid optional keyword arguments are identical to those of self-signatures for :py:meth:`PGPKey.certify`.
    Any such keyword arguments are ignored if selfsign is ``False``



In [8]:
# this is similar to GnuPG 2.1.x defaults, with no expieration or preferred keyserver
# cand add an expiration argument using from datetime import timedelta
# key_expires=timedelta(days=365) for a one year expiration
key.add_uid(uid=uid, usage={KeyFlags.Sign, KeyFlags.EncryptCommunications, KeyFlags.EncryptStorage},
           hashes=[HashAlgorithm.SHA256, HashAlgorithm.SHA384, HashAlgorithm.SHA512, HashAlgorithm.SHA224],
           ciphers=[SymmetricKeyAlgorithm.AES256, SymmetricKeyAlgorithm.AES192, SymmetricKeyAlgorithm.AES128],
           compression=[CompressionAlgorithm.ZLIB, CompressionAlgorithm.BZ2, CompressionAlgorithm.ZIP, CompressionAlgorithm.Uncompressed])

In [9]:
# checking some key parameters
key.is_public

False

In [10]:
key.is_protected

False

- It is usually recommended to password protect private keys

In [11]:
# protecting a private key
key.protect('P@ssw0rd', SymmetricKeyAlgorithm.AES256, HashAlgorithm.SHA256)

In [12]:
# key is now protected
key.is_protected

True

- Unlocking a key is required to use it
- Always use the with block here
- Key remains protected the entire time, it just becomes usable

- To Encrypting a message
    - Requires a public key, decryption is done with a private key
    - Add the public key to the private key created earlier
    - I tried to do this much later in this tutorial, and I got some errors
        - It appears you can only run `with key.unlock()` once before throwing strange exceptions
    - Be sure to add public keys by this point

In [13]:
# add a public key for encryption as a subkey
# you need to unlock a protected key to do this
rsa_pub_key = key.pubkey

In [14]:
rsa_pub_key.is_public

True

In [15]:
str(rsa_pub_key)

'-----BEGIN PGP PUBLIC KEY BLOCK-----\n\nxsFNBF7VTZoBEACmdYezYCk80281xiOBhpEWG1fqr2S8JMHNoUF/vI3YTrHl1gbO\nFHYXnN3K+KeZxsJIp4WMBqvWwlv/0DggXBuZdFYKPZycEIPPAjMRTL61W8TLsbtr\nMrjNf6MKABCGcxwIDqlBk90nvb1oxGnIZ0TV7bdtnADFsSfQUZWBoUQz02R9k2Ay\nDNfBeI3nLNYn6/5vooAcvQbIW69uSpsMolSzBVCC8yq0pFwApbCsntTCXDGwkjq+\nEP/QcxlEWbe9em5xlhdznldQq8WCkuHk6zqOyu4ny/Xgfnc5NpVfV3TMA0FeP+CX\nlseo3moz1h6WVQB/3tW5tlFugeWXsiJrnCkDjOCJmjCXmC07W0bKllkYbcWMkC2x\nrzGLE/69gBLSM63iKO0DK0iFs35KnPFyArMtH/Vc/d5/h8YTTkcLqI8YLopxfXK0\n6IhK6Syd9RqiDithOSM9VfPpsvD0eP2MRxHYygn1/IqFuQWt56z4r6vbjXQZn+Vi\np64uyxMl351GsDO+SxKJJUDnCm4C5qPsuqoatvXCPjD0NldaBYKcByp9jxHwAlEL\nPDkgDf6aaNDF4NxGLlPff6P9Fdzg7Mft6ifJFrAc+BFXyi8SWQ1hwMvVVRi1MLgr\npSpPJw36eF1jdggdGqXLZuzRIk1CJ0So+zeOSplGjnnm4qZoo6btAHVoRwARAQAB\nzT9DaHVjayBUdWNrZXIgKFNDIHNpZGUgdXNlcm5hbWUpIDxjaHVjay50dWNrZXJA\nc2FudGVlY29vcGVyLmNvbT7CwXMEEwEIAB0FAl7VTZsCGw4ECwkIBwUVCAkKCwUW\nAgMBAAIeAQAKCRAv5ej1uuhpfhdjD/97+pulzChkUHglk2fSzccnkI3VWdKPsXRJ\nuN/W2DYP8FpBbRJUhvDZKOtRZjQi4/hEQ63

In [16]:
# sign the key to be able to retrieve it
# not necessary if using file to store the key

# with key.unlock('P@ssw0rd'):
#     key.sign('RMS')

## Before loading into a keyring, you may need to add an identifier to the key to retrieve it
- See the docs on `help(kr)` below

- Loading keys into a keyring
    - good if intending to maintain multiple keys in memory for extended periods

In [17]:
# import os
# from glob import glob

In [18]:
# loading keys into a keyring
# kr = pgpy.PGPKeyring(glob(os.path.expanduser('~/.gnupg/*ring.gpg')))

In [19]:
# load the key
# kr.load(key)

#### Retrieving a key from a keyring
- Haven't been able to implement this
- Will try from file

In [20]:
# kr.fingerprints()

In [21]:
# kr.key('93D1 578B C5FC B1F7 AEBC  F28F F099 8F1F 4D54 FFD3')

## Store key in a file

In [22]:
# private key is ASCII armored
# this key is also protected, so should be secure
with open('pv_key_file.asc', 'w') as pv_key_file:
    pv_key_file.write(str(key))

## Load key from file

In [23]:
key2, _ = pgpy.PGPKey.from_file('pv_key_file.asc')

In [24]:
key2.is_public

False

In [25]:
key2.is_protected

True

In [26]:
# add the public key to the private key as a subkey
# may not need to do this, since we can get the pubkey straight from the protected key
# see above


# with key.unlock('P@ssw0rd') as ukey:
    
#     # add the public key to allow encryption of messages
#     ukey.add_subkey(rsa_pub_key, usage={KeyFlags.Authentication, KeyFlags.EncryptStorage, KeyFlags.EncryptCommunications})

- Exporting keys in OpenPGP compliant binary or ASCII-armored formats

In [27]:
# binary
keybytes = bytes(key)

# ASCII armored
keystr = str(key)

In [28]:
keybytes

b'\xc5\xc6\x86\x04^\xd5M\x9a\x01\x10\x00\xa6u\x87\xb3`)<\xd3o5\xc6#\x81\x86\x91\x16\x1bW\xea\xafd\xbc$\xc1\xcd\xa1A\x7f\xbc\x8d\xd8N\xb1\xe5\xd6\x06\xce\x14v\x17\x9c\xdd\xca\xf8\xa7\x99\xc6\xc2H\xa7\x85\x8c\x06\xab\xd6\xc2[\xff\xd08 \\\x1b\x99tV\n=\x9c\x9c\x10\x83\xcf\x023\x11L\xbe\xb5[\xc4\xcb\xb1\xbbk2\xb8\xcd\x7f\xa3\n\x00\x10\x86s\x1c\x08\x0e\xa9A\x93\xdd\'\xbd\xbdh\xc4i\xc8gD\xd5\xed\xb7m\x9c\x00\xc5\xb1\'\xd0Q\x95\x81\xa1D3\xd3d}\x93`2\x0c\xd7\xc1x\x8d\xe7,\xd6\'\xeb\xfeo\xa2\x80\x1c\xbd\x06\xc8[\xafnJ\x9b\x0c\xa2T\xb3\x05P\x82\xf3*\xb4\xa4\\\x00\xa5\xb0\xac\x9e\xd4\xc2\\1\xb0\x92:\xbe\x10\xff\xd0s\x19DY\xb7\xbdznq\x96\x17s\x9eWP\xab\xc5\x82\x92\xe1\xe4\xeb:\x8e\xca\xee\'\xcb\xf5\xe0~w96\x95_Wt\xcc\x03A^?\xe0\x97\x96\xc7\xa8\xdej3\xd6\x1e\x96U\x00\x7f\xde\xd5\xb9\xb6Qn\x81\xe5\x97\xb2"k\x9c)\x03\x8c\xe0\x89\x9a0\x97\x98-;[F\xca\x96Y\x18m\xc5\x8c\x90-\xb1\xaf1\x8b\x13\xfe\xbd\x80\x12\xd23\xad\xe2(\xed\x03+H\x85\xb3~J\x9c\xf1r\x02\xb3-\x1f\xf5\\\xfd\xde\x7f\x87\xc6\x13NG\x0b\xa8\x8

In [ ]:
# view the private key block
# keystr

- Creating New Messages
    - This is the total process for encryption
    - Note: **messages are not encrypted and must be encrypted**

In [30]:
# pack (read contents of) a file into a message
# PGPMessage will store the basename of the file and the time it was last modified
message = pgpy.PGPMessage.new('Sample_XML_Data/RMS_BATCH_CANDIDATE_EXPORT_CLIENT_SANTEECOOPER_Sample - Copy', file=True)

In [31]:
str(message)

'-----BEGIN PGP MESSAGE-----\n\nyNerAe1d65LiOJbe/TsR8w4KJmKiO6IhfeE+2UyRxplJF7fBkNX5ZwknKBNPGZu2\nTWXlPs3+2xfYJ9i/+wr7LrtHEr4BBpOVQBalro4qLB3Jx9LRpyOdc6T/+b9/+df8\nfz1c9tva6Ko+UG5HSr3TaDbqA3Wk/t7r9gcjpdVUO4ORVu8MVFXpdntqf6Tps7mJ\nURYp9vzl3/7z+n8v//51ZqIv2HEN2/o1I+aEDMLW2J4Y1tOvmeHgOlvO/L12eWeQ\n/Ibu6erX8VS3njCCcpZb/eoa6FeUmXrevHpx8fz8nHuWc7bzdCEJgnjxe7uljad4\npmcNy/V0a4wztT//6VJx8ARbnqGbLn00DXiqjek/I1e3PIzHtj3HzuXFMg+o7hiP\ntXxOvrzwHyD9YqU29evcdjzCag2q1q2JMdE9jMbwq9n4NSOXK0IpU/sTQjSzjx8V\ne4JdkhBPQtAI+NdME1rvpl8fNLudDJpBuzwaeNKFpspc0EouVmu5vDYc1+voM1xT\npo7hevZ8Sr4kTCZEbWMyMTF9vNINE79cXkSSCEVLX5LXTRNblxfBM8nUFo+Pxtfa\n5cXyB0mrTyYOdl2xJkol9MmwLOjCqT5DPVMf48uLIDtCK9WCdCmaLofpMmsqw3up\ntW1r/NkF0XEs2jUkjTLjQQP3HPuLAR28bDZNydQ0e+FNkaI7tmlYOvAapaMlezZI\nhUmariZV8kXx8iKSQt9rLyzPeVlWOtTqmdrQMjw8QbQyF7hgFEtqy9PHXht7U3uy\nLLPsplt7hntT28J+R7Ni9EViUIv/XtISDtbpYzkP8hY8sczOYvaAnVpZKmZLspS/\nvFgmsFz1q4ctKp6XF+FvltdzjJkefJCYqd2Tj1imMoGKsXr5yXY+JzG+le8kpl/N\nbKa2wmiMtcu2/QBy/P7aWMjUOvYK5yu8Xl7rX99pG0c5

- To Encrypting a message
    - Requires a public key, decryption is done with a private key
    - Add the public key to the private key created earlier
    - I tried to do this much later in this tutorial, and I got some errors
        - It appears you can only run `with key.unlock()` once before throwing strange exceptions
    - Be sure to add public keys by this point

In [32]:
# you can get a public key from a private key by accessing the pubkey attribute
rsa_pub_key = key.pubkey

#### Write public key to file

In [33]:
# can distribute this public key for encryption of data
with open('pub_key_file.asc', 'w') as pub_key_file:
    pub_key_file.write(str(rsa_pub_key))

#### Load public key from file

In [34]:
rsa_pub_key2, _ = pgpy.PGPKey.from_file('pub_key_file.asc')

In [35]:
rsa_pub_key2.is_public

True

In [36]:
# this returns a new PGPMessage that contains an encrypted form of the original message
# using the key created above

encrypted_message = rsa_pub_key.encrypt(message)

## NOTES:
- May want to modify the 'w' write parameter below in an app

In [37]:
# write the encrypted message to a file
# could also write the bytes version to a file (bytes(file_message))


out_file_path = 'Sample_XML_Data/encrypted_message.txt'
with open(out_file_path, 'w') as output_file:
    output_file.write(str(encrypted_message))

- Loading existing messages
    - a message from a file for example

In [38]:
# read in the message from an encrypted file
message_from_file = pgpy.PGPMessage.from_file('Sample_XML_Data/encrypted_message.txt')

In [39]:
# view the encrypted information
str(message_from_file)

'-----BEGIN PGP MESSAGE-----\n\nwcFMAy/l6PW66Gl+AQ/+JF61YddJN4JTERDCekl3jrV6cR8yKEjkcmxRJOyhRGqg\n7GiZSOM/NuY9YrjVYeFNxk/4VaZJupUSfldMIrc+bd8/MwLw6pu3FiwrSczfplIN\nQi5t29Qbquevfiw6S1VDE6njGj9kwgDTwZ27PcQVnxrEzS4WQB1Else5NvO5WByC\n1Xvu3lKXJgVcSJ1yjrW6aX7aE7C1rurfrNSfUcokArANGz1/18M6UN7kW8EeenS1\nE40N0UsBkY4mk1nhGkByVmoxrRkmXVNkzXqvQf6Ys22Dc8dZTW9vQlEXzCdbv5Yx\nbvPivXp6tc42nLqcCqf+PBl1m2EsWIhmpj3v8V314YvM2AA994mDAYv82g7kvWw0\ncekJcDgtdF9Ie7lxjHP8dHRo1VHZ2gBLOR3ns9xHywR+ToJyW+vGdLPviXwXNOXI\nZ+zwdULrF1bQU/RYcuvruGTkxfa+7oNM/uIxoKgU3r6Xzt8uGP01tdDpcQxdHFC0\nOzcCvr0H/FX+Cec4ELgrpHTPBlhNdb1etuNTDdg3TKLhiN221YtSCv89pLa+aybV\nK85iJdnl4mBaE5sbgQWD2hMRk8/mguHDHaOVIwfpjpmt4cExSu11VUA00TXNH3NN\nAxzoB+ymTH5O1wH8LfR3DiZ9+EbZpb7+FnvZh7mdCg/P28082Q2WgxecY7UPQnvS\n19cBES1548XHiQSoDDxSpM2rMKdycT5JMsGvP+zBJWqqcWS2mhRwsxA5S/C7nTSW\nk15+o9W9JxHd43HcsbfyuSXeqdK/0rJhpEhEyBoGIF7WZM9cwDYHVHJv1AA3vv/p\nAmE66olEzi2Pk8WIPkvzqSBHfvN4pm8ROp6KbYlHpZQwioDnspTDfVyVxsoDBwOY\nV62uriXxSizKyQXPm3Gqo9pu3HocstDeeUd/zIP/Sj7O

In [40]:
help(message_from_file)

Help on PGPMessage in module pgpy.pgp object:

class PGPMessage(pgpy.types.Armorable, pgpy.types.PGPObject)
 |  Method resolution order:
 |      PGPMessage
 |      pgpy.types.Armorable
 |      pgpy.types.PGPObject
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __bytearray__(self)
 |      Returns the contents of concrete subclasses in a binary format that can be understood by other OpenPGP
 |      implementations
 |  
 |  __copy__(self)
 |  
 |  __init__(self)
 |      PGPMessage objects represent OpenPGP message compositions.
 |      
 |      PGPMessage implements the `__str__` method, the output of which will be the message composition in
 |      OpenPGP-compliant ASCII-armored format.
 |      
 |      PGPMessage implements the `__bytes__` method, the output of which will be the message composition in
 |      OpenPGP-compliant binary format.
 |      
 |      Any signatures within the PGPMessage that are marked as being non-exportable will not be included in the output


In [41]:
message_from_file.is_encrypted

True

In [42]:
# decrypt the message
# must unlock the key to decrypt
with key.unlock('P@ssw0rd'):
    decrypted_message = key.decrypt(message_from_file)

In [43]:
str(decrypted_message)

'-----BEGIN PGP MESSAGE-----\n\nyNfYAe1dvZPjOHa306tyYJerLkXpqq52q1Zqfuj7enWjptjdutXXidLMdmIVW0K3\neEORWpKanva/4X/AgaucOnDkwIFTh06dO3RuB/YDIH5JokT1tKQeDXa2ZkTgAXwE\nHn54wHsP+Pf/+7M/z//z/WW/rY2u6gPldqTUO41moz5QR+rPvW5/MFJaTbUzGGn1\nzkBVlW63p/ZHmj6bmxhlkWLPn//mH6//8/L3n2cm+oQd17CtHzNiTsggbI3tiWE9\n/pgZDq6z5czva5fvDZLf0D1d/Tye6tYjRlDOcqufXQP9iDJTz5tXLy6enp5yT3LO\ndh4vJEEQL35ut7TxFM/0rGG5nm6Ncab2F7+6VBw8wZZn6KZLH00Dnmpj+s/I1S0P\n47Ftz7FzebHMA6r3jMdaPidfXvgPkH6xUpv6eW47HmG1BlXr1sSY6B5GY/jVbPyY\nkcsVoZSp/QohmtnHD4o9wS5JiCchaAT8Y6YJrXfTrw+a3U4GzaBdHgw86UJTZS5o\nJRertVxeG47rdfQZrilTx3A9ez4lXxImE6K2MZmYmD5e6YaJny8vIkmEoqUvyeum\nia3Li+CZZGqLhwfjc+3yYvmDpNUnEwe7rlgTpRL6YFgWdOFUn6GeqY/x5UWQHaGV\nakG6FE2Xw3SZNZXhPdfatjX+6ILoOBbtGpJGmfGggXuO/cmADl42m6Zkapq98KZI\n0R3bNCwdeI3S0ZI9G6TCJE1Xkyr5onh5EUmh77UXluc8LysdavVMbWgZHp4gWpkL\nXDCKJbXl6WOvjb2pPVmWWXbTrT3DvaltYb+jWTH6IjGoxX8vaQkH6/SxnAd5C55Y\nZmcxu8dOrSwVsyVZyl9eLBNYrvrZwxYVz8uL8DfL6znGTA8+SMzU7shHLFOZQMVY\nvfxgOx+TGN/KdxLTL2Y2U1thNMbaZdu+Bzl+e20sZGod

In [44]:
help(decrypted_message)

Help on PGPMessage in module pgpy.pgp object:

class PGPMessage(pgpy.types.Armorable, pgpy.types.PGPObject)
 |  Method resolution order:
 |      PGPMessage
 |      pgpy.types.Armorable
 |      pgpy.types.PGPObject
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __bytearray__(self)
 |      Returns the contents of concrete subclasses in a binary format that can be understood by other OpenPGP
 |      implementations
 |  
 |  __copy__(self)
 |  
 |  __init__(self)
 |      PGPMessage objects represent OpenPGP message compositions.
 |      
 |      PGPMessage implements the `__str__` method, the output of which will be the message composition in
 |      OpenPGP-compliant ASCII-armored format.
 |      
 |      PGPMessage implements the `__bytes__` method, the output of which will be the message composition in
 |      OpenPGP-compliant binary format.
 |      
 |      Any signatures within the PGPMessage that are marked as being non-exportable will not be included in the output


In [45]:
type(decrypted_message.message)

bytearray

In [46]:
contents = decrypted_message.message.decode()

In [47]:
# remove hard returns that were added in this case
contents = contents.replace('\r', '')

In [48]:
# write the decrypted contents to a file
# could also write the bytes version to a file (bytes(file_message))


final_file_path = 'Sample_XML_Data/decrypted_data'
with open(final_file_path, 'w') as output_file:
    output_file.write(contents)

In [50]:
# miscellaneous
type(key) == pgpy.pgp.PGPKey

True